## Train model
Danh sách class: <br>
    0. Correct <br>
    1. Chân quá hẹp <br >
    2. Chân quá rộng <br >
    3. Khoảng cách giữa 2 đầu gối quá nhỏ <br >
    4. Xuống quá xâu <br >
    5. Lỗi gập gập lưng <br >

In [73]:
import mediapipe as mp
import cv2
import pandas as pd
import pickle
import numpy as np
import csv
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import clear_output

import warnings
warnings.filterwarnings('ignore')

# Drawing helpers
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# 1. Khởi tạo keypoints và các phương thức 
Các keypoints cần lấy :
    "NOSE",
    "LEFT_SHOULDER",
    "RIGHT_SHOULDER",
    "LEFT_HIP",
    "RIGHT_HIP",
    "LEFT_KNEE",
    "RIGHT_KNEE",
    "LEFT_ANKLE",
    "RIGHT_ANKLE",

Dataframe gồm label và các features là toạ độ keypoints gồm 4 thành phần: x, y, z, visibility
- x, y: tọa độ x, y của keypoints đã được chuẩn hóa min max scale
- z: độ sâu của điểm ảnh theo điểm giữa hông 
- visibility: biểu thị khả năng có thể nhìn thấy hoặc che khuất
 

In [74]:
#Chọn các điểm cần lấy
Important_kp = [
    "NOSE",
    "LEFT_SHOULDER",
    "RIGHT_SHOULDER",
    "LEFT_HIP",
    "RIGHT_HIP",
    "LEFT_KNEE",
    "RIGHT_KNEE",
    "LEFT_ANKLE",
    "RIGHT_ANKLE",
]

#Tạo header cho dataframe
header = ["label"]

for kp in Important_kp:
    header.extend([f"{kp}_x", f"{kp}_y", f"{kp}_z", f"{kp}_visibility"])

header

['label',
 'NOSE_x',
 'NOSE_y',
 'NOSE_z',
 'NOSE_visibility',
 'LEFT_SHOULDER_x',
 'LEFT_SHOULDER_y',
 'LEFT_SHOULDER_z',
 'LEFT_SHOULDER_visibility',
 'RIGHT_SHOULDER_x',
 'RIGHT_SHOULDER_y',
 'RIGHT_SHOULDER_z',
 'RIGHT_SHOULDER_visibility',
 'LEFT_HIP_x',
 'LEFT_HIP_y',
 'LEFT_HIP_z',
 'LEFT_HIP_visibility',
 'RIGHT_HIP_x',
 'RIGHT_HIP_y',
 'RIGHT_HIP_z',
 'RIGHT_HIP_visibility',
 'LEFT_KNEE_x',
 'LEFT_KNEE_y',
 'LEFT_KNEE_z',
 'LEFT_KNEE_visibility',
 'RIGHT_KNEE_x',
 'RIGHT_KNEE_y',
 'RIGHT_KNEE_z',
 'RIGHT_KNEE_visibility',
 'LEFT_ANKLE_x',
 'LEFT_ANKLE_y',
 'LEFT_ANKLE_z',
 'LEFT_ANKLE_visibility',
 'RIGHT_ANKLE_x',
 'RIGHT_ANKLE_y',
 'RIGHT_ANKLE_z',
 'RIGHT_ANKLE_visibility']

In [75]:
from math import sqrt

# rescale frame 1/2
def rescale_frame(frame, percent=50):
    '''
    Rescale a frame to a certain percentage compare to its original frame
    '''
    width = int(frame.shape[1] * percent/ 100)
    height = int(frame.shape[0] * percent/ 100)
    dim = (width, height)
    return cv2.resize(frame, dim, interpolation =cv2.INTER_AREA)

#Tính khoảng cách giữa 2 điểm trong không gian 2D
def calculate_distance(pointX, pointY) -> float:
    '''
    Calculate a distance between 2 points
    '''

    x1, y1 = pointX
    x2, y2 = pointY

    return sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

#Tính góc giữa 3 điểm trong không gian 3D
def calculate_angle(pointA, pointB, pointC) -> float:
    """
    Tính góc giữa ba điểm trong không gian 3D (đơn vị: độ)
    a, b, c là danh sách chứa tọa độ 3D: [x, y, z]
    """

    #Độ sâu ảnh
    t = 33;
    A = np.array([pointA[0], pointA[1], pointA[2]/2])  # (xA, yA, zA)
    B = np.array([pointB[0], pointB[1], pointB[2]/2 ])  # (xB, yB, zB)
    C = np.array([pointC[0], pointC[1], pointC[2]/2])  # (xC, yC, zC)

    # Vector BA và BC
    BA = A - B
    BC = C - B

    # Tính tích vô hướng
    dot_product = np.dot(BA, BC)
    
    # Tính độ dài vector
    norm_BA = np.linalg.norm(BA)
    norm_BC = np.linalg.norm(BC)

    # Tính góc bằng công thức cos(theta) = (A.B) / (|A| * |B|)
    cos_theta = dot_product / (norm_BA * norm_BC)
    
    # Chuyển từ radian sang độ
    angle = np.degrees(np.arccos(np.clip(cos_theta, -1.0, 1.0)))

    return angle

#Đọc dữ liệu từ frame trả về points
def extract_important_keypoints(results) -> list:
    '''
    Extract important keypoints from mediapipe pose detection
    '''
    landmarks = results.pose_landmarks.landmark

    data = []
    for lm in Important_kp:
        keypoint = landmarks[mp_pose.PoseLandmark[lm].value]
        data.append([keypoint.x, keypoint.y, keypoint.z, keypoint.visibility])
    
    return np.array(data).flatten().tolist()


In [76]:
#Lỗi 1 - 2: ratio = feet/shoulder
Min_radio_feet_shoulder = 0.59
Max_radio_feet_shoulder = 1.11

#Lỗi 3 : ratio = knee/feet
Min_up_radio_knee_feet = 0.74 
Min_middle_ratio_knee_feet = 0.81 
Min_down_ratio_knee_feet = 1.31

#Lỗi 4: knee angle < 50
Min_knee_angle = 58

#Lỗi 5: ratio = knee_angle/hip_angle
Max_ratio_knee_hip = 1


In [77]:
df = pd.DataFrame(columns=header)
df_test = pd.DataFrame(columns=header)
df_va = pd.DataFrame(columns=header)
df 

,label,NOSE_x,NOSE_y,NOSE_z,NOSE_visibility,LEFT_SHOULDER_x,LEFT_SHOULDER_y,LEFT_SHOULDER_z,LEFT_SHOULDER_visibility,RIGHT_SHOULDER_x,...,RIGHT_KNEE_z,RIGHT_KNEE_visibility,LEFT_ANKLE_x,LEFT_ANKLE_y,LEFT_ANKLE_z,LEFT_ANKLE_visibility,RIGHT_ANKLE_x,RIGHT_ANKLE_y,RIGHT_ANKLE_z,RIGHT_ANKLE_visibility


# 2. Hàm duyệt data lỗi chân quá hẹp

In [78]:
#Xử lý từng video để tính khoảng cách chân, vai
def detection_1_error(Video_folder, Video_name, type):
    
    Cap = cv2.VideoCapture(f"{Video_folder}/{Video_name}")

    images = []  # Danh sách lưu hình ảnh
    max_images = 10  # Số hình ảnh tối đa hiển thị cùng lúc

    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while Cap.isOpened():
            ret, image = Cap.read()

            if not ret:
                break

            # Chuyển ảnh sang RGB
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = rescale_frame(image, percent=50)
            image.flags.writeable = True  # Make the image writable

            #trích xuất kpkp
            results = pose.process(image)

            # Kiểm tra có nhận được keypoint không
            if not results.pose_landmarks:
                continue
            
            #khởi tạo biến tính khoảng cách
            shoulder_width = feet_width = None
            
            try:
                # Lấy kp từ frame
                # row = extract_important_keypoints(results)
                # X = pd.DataFrame([row], columns=header[1:])

                # Tinhs toán và so sánh khoảng cách vai, hông, chân
                landmarks = results.pose_landmarks.landmark
                # Khoảng cách giữa 2 vai
                left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]

                shoulder_width = calculate_distance(left_shoulder, right_shoulder)

                # Khoảng cách giữa 2 chân (sửa từ foot_index thành ankle)
                left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
                right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]

                feet_width = calculate_distance(left_ankle, right_ankle)


                # Kiểm tra lỗi
                row = []
                if(feet_width/shoulder_width < Min_radio_feet_shoulder):
                    row.append(1)
                # else:
                #     row.append(0)
                
                # Lưu dữ liệu
                for lm in Important_kp:
                        keypoint = landmarks[mp_pose.PoseLandmark[lm].value]
                        row.extend([keypoint.x, keypoint.y, keypoint.z, keypoint.visibility])

                # row.extend([Video_name])

                if len(row) == len(df.columns):
                    if type == "train":
                        df.loc[len(df)] = row
                    elif type == "test":
                        df_test.loc[len(df_test)] = row
                    elif type == "validate":
                        df_va.loc[len(df_va)] = row
                else:
                    print(f"Lỗi! row có {len(row)} phần tử nhưng df có {len(df.columns)} cột.")

                # Nền
                cv2.rectangle(image, (0, 0), (500, 60), (245, 117, 16), -1)

                # Display feet distance
                cv2.putText(image, "FEET", (15, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(feet_width, 2)), (10, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                # Display shoulder distance
                cv2.putText(image, "SHOULDER", (95, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(shoulder_width, 2)), (90, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                # Display label
                cv2.putText(image, "LABEL", (200, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(row[0], 2)), (200, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

            except Exception as e:
                print(f"Error: {e}")

            # Draw landmarks and connections
            # mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, mp_drawing.DrawingSpec(color=(244, 117, 66), thickness=2, circle_radius=4), mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

            images.append(image)

            # Khi đủ 10 ảnh, hiển thị tất cả cùng lúc
            # if len(images) == max_images:
            #     fig, axes = plt.subplots(5, 2, figsize=(32, 18))  # Tạo grid 2x5
            #     for i, ax in enumerate(axes.flat):
            #         ax.imshow(images[i])
            #         ax.set_aspect('auto')
            #         ax.axis("off")  # Ẩn trục tọa độ
            #     plt.show()
            #     images = []  # Reset danh sách sau khi hiển thị

        Cap.release()

In [79]:
import os

video_folder = "Data/Train/Chan_qua_hep"
video_files = [f for f in os.listdir(video_folder) if f.endswith(".mp4")]

for video_file in video_files:
    detection_1_error(video_folder, video_file, "train")

video_folder = "Data/Test/Chan_qua_hep"
video_files = [f for f in os.listdir(video_folder) if f.endswith(".mp4")]

for video_file in video_files:
    detection_1_error(video_folder, video_file, "test")

video_folder = "Data/Validation/Chan_qua_hep"
video_files = [f for f in os.listdir(video_folder) if f.endswith(".mp4")]

for video_file in video_files:
    detection_1_error(video_folder, video_file, "validate")


Lỗi! row có 36 phần tử nhưng df có 37 cột.
Lỗi! row có 36 phần tử nhưng df có 37 cột.
Lỗi! row có 36 phần tử nhưng df có 37 cột.
Lỗi! row có 36 phần tử nhưng df có 37 cột.
Lỗi! row có 36 phần tử nhưng df có 37 cột.
Lỗi! row có 36 phần tử nhưng df có 37 cột.
Lỗi! row có 36 phần tử nhưng df có 37 cột.
Lỗi! row có 36 phần tử nhưng df có 37 cột.
Lỗi! row có 36 phần tử nhưng df có 37 cột.
Lỗi! row có 36 phần tử nhưng df có 37 cột.
Lỗi! row có 36 phần tử nhưng df có 37 cột.
Lỗi! row có 36 phần tử nhưng df có 37 cột.
Lỗi! row có 36 phần tử nhưng df có 37 cột.
Lỗi! row có 36 phần tử nhưng df có 37 cột.
Lỗi! row có 36 phần tử nhưng df có 37 cột.
Lỗi! row có 36 phần tử nhưng df có 37 cột.
Lỗi! row có 36 phần tử nhưng df có 37 cột.
Lỗi! row có 36 phần tử nhưng df có 37 cột.
Lỗi! row có 36 phần tử nhưng df có 37 cột.
Lỗi! row có 36 phần tử nhưng df có 37 cột.
Lỗi! row có 36 phần tử nhưng df có 37 cột.
Lỗi! row có 36 phần tử nhưng df có 37 cột.
Lỗi! row có 36 phần tử nhưng df có 37 cột.
Lỗi! row có

In [80]:
df 

,label,NOSE_x,NOSE_y,NOSE_z,NOSE_visibility,LEFT_SHOULDER_x,LEFT_SHOULDER_y,LEFT_SHOULDER_z,LEFT_SHOULDER_visibility,RIGHT_SHOULDER_x,...,RIGHT_KNEE_z,RIGHT_KNEE_visibility,LEFT_ANKLE_x,LEFT_ANKLE_y,LEFT_ANKLE_z,LEFT_ANKLE_visibility,RIGHT_ANKLE_x,RIGHT_ANKLE_y,RIGHT_ANKLE_z,RIGHT_ANKLE_visibility
0,1.0,0.566628,0.215825,-0.179164,0.999983,0.603728,0.310659,0.009625,0.999969,0.515302,...,-0.018046,0.994093,0.583166,0.914074,0.129855,0.989608,0.550327,0.910379,0.112435,0.993049
1,1.0,0.566143,0.215827,-0.166491,0.999982,0.603697,0.310681,0.032897,0.999968,0.515275,...,-0.027877,0.993640,0.583169,0.915207,0.129954,0.988759,0.549618,0.911830,0.104301,0.992580
2,1.0,0.565114,0.216485,-0.167129,0.999982,0.603651,0.311274,0.027464,0.999966,0.515112,...,-0.026802,0.993451,0.583109,0.915970,0.136486,0.988204,0.549167,0.912555,0.112636,0.992348
3,1.0,0.564665,0.218071,-0.178488,0.999981,0.603564,0.312767,0.024608,0.999965,0.514824,...,-0.027260,0.993313,0.582990,0.915961,0.139435,0.987626,0.549021,0.912643,0.116197,0.992105
4,1.0,0.564257,0.219991,-0.219889,0.999980,0.603501,0.314149,-0.001277,0.999963,0.514392,...,-0.022762,0.993191,0.582212,0.915911,0.152998,0.986772,0.548943,0.912549,0.134366,0.991728
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3765,1.0,0.480709,0.151462,-0.304861,0.999981,0.526945,0.260478,-0.084773,0.999830,0.428703,...,0.031203,0.996730,0.491099,0.890934,0.208882,0.990742,0.455111,0.893892,0.205204,0.995438
3766,1.0,0.480790,0.151496,-0.304818,0.999981,0.526946,0.260702,-0.084558,0.999829,0.428731,...,0.031312,0.996726,0.491095,0.891209,0.208973,0.990803,0.455155,0.894002,0.205221,0.995427
3767,1.0,0.480804,0.151605,-0.300575,0.999981,0.526949,0.260844,-0.081109,0.999831,0.428785,...,0.031301,0.996716,0.491092,0.891493,0.208588,0.990877,0.455188,0.894193,0.204733,0.995404
3768,1.0,0.480813,0.151777,-0.295606,0.999981,0.526992,0.260947,-0.077437,0.999832,0.428822,...,0.031008,0.996712,0.491091,0.891643,0.207617,0.990930,0.455219,0.894289,0.202291,0.995391


# 2. Lỗi chân rộng


In [81]:
#Xử lý từng video để tính khoảng cách chân, vai
def detection_2_error(Video_folder, Video_name, type):
    
    Cap = cv2.VideoCapture(f"{Video_folder}/{Video_name}")

    images = []  # Danh sách lưu hình ảnh
    max_images = 10  # Số hình ảnh tối đa hiển thị cùng lúc

    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while Cap.isOpened():
            ret, image = Cap.read()

            if not ret:
                break

            # Chuyển ảnh sang RGB
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = rescale_frame(image, percent=50)
            image.flags.writeable = True  # Make the image writable

            #trích xuất kpkp
            results = pose.process(image)

            # Kiểm tra có nhận được keypoint không
            if not results.pose_landmarks:
                continue
            
            #khởi tạo biến tính khoảng cách
            shoulder_width = feet_width = None
            
            try:
                # Lấy kp từ frame
                # row = extract_important_keypoints(results)
                # X = pd.DataFrame([row], columns=header[1:])

                # Tinhs toán và so sánh khoảng cách vai, hông, chân
                landmarks = results.pose_landmarks.landmark
                # Khoảng cách giữa 2 vai
                left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]

                shoulder_width = calculate_distance(left_shoulder, right_shoulder)

                # Khoảng cách giữa 2 chân
                left_foot_index = [landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].x, landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].y]
                right_foot_index = [landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].y]

                feet_width = calculate_distance(left_foot_index, right_foot_index)

                # Kiểm tra lỗi
                row = []
                if(feet_width/shoulder_width > Max_radio_feet_shoulder):
                    row.append(2)
                # else:
                #     row.append(0)

                # Lưu dữ liệu
                for lm in Important_kp:
                        keypoint = landmarks[mp_pose.PoseLandmark[lm].value]
                        row.extend([keypoint.x, keypoint.y, keypoint.z, keypoint.visibility])
                
                # row.extend([Video_name])
                        
                if len(row) == len(df.columns):
                    if type == "train":
                        df.loc[len(df)] = row
                    elif type == "test":
                        df_test.loc[len(df_test)] = row
                    elif type == "validate":
                        df_va.loc[len(df_va)] = row
                else:
                    print(f"Lỗi! row có {len(row)} phần tử nhưng df có {len(df.columns)} cột.")

                # Nền
                cv2.rectangle(image, (0, 0), (500, 60), (245, 117, 16), -1)

                # Display feet distance
                cv2.putText(image, "FEET", (15, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(feet_width, 2)), (10, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                # Display shoulder distance
                cv2.putText(image, "SHOULDER", (95, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(shoulder_width, 2)), (90, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                # Display label
                cv2.putText(image, "LABEL", (200, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(row[0], 2)), (200, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

            except Exception as e:
                print(f"Error: {e}")

            # Draw landmarks and connections
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, mp_drawing.DrawingSpec(color=(244, 117, 66), thickness=2, circle_radius=4), mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

            images.append(image)

            # Khi đủ 10 ảnh, hiển thị tất cả cùng lúc
            # if len(images) == max_images:
            #     fig, axes = plt.subplots(5, 2, figsize=(32, 18))  # Tạo grid 2x5
            #     for i, ax in enumerate(axes.flat):
            #         ax.imshow(images[i])
            #         ax.set_aspect('auto')
            #         ax.axis("off")  # Ẩn trục tọa độ
            #     plt.show()
            #     images = []  # Reset danh sách sau khi hiển thị

        Cap.release()

In [82]:
import os

video_folder = "Data/Train/Chan_qua_rong"
video_files = [f for f in os.listdir(video_folder) if f.endswith(".mp4")]

for video_file in video_files:
    detection_2_error(video_folder, video_file, "train")

video_folder = "Data/Test/Chan_qua_rong"
video_files = [f for f in os.listdir(video_folder) if f.endswith(".mp4")]

for video_file in video_files:
    detection_2_error(video_folder, video_file, "test")

video_folder = "Data/Validation/Chan_qua_rong"
video_files = [f for f in os.listdir(video_folder) if f.endswith(".mp4")]

for video_file in video_files:
    detection_2_error(video_folder, video_file, "validate")

Lỗi! row có 36 phần tử nhưng df có 37 cột.


In [83]:
df

,label,NOSE_x,NOSE_y,NOSE_z,NOSE_visibility,LEFT_SHOULDER_x,LEFT_SHOULDER_y,LEFT_SHOULDER_z,LEFT_SHOULDER_visibility,RIGHT_SHOULDER_x,...,RIGHT_KNEE_z,RIGHT_KNEE_visibility,LEFT_ANKLE_x,LEFT_ANKLE_y,LEFT_ANKLE_z,LEFT_ANKLE_visibility,RIGHT_ANKLE_x,RIGHT_ANKLE_y,RIGHT_ANKLE_z,RIGHT_ANKLE_visibility
0,1.0,0.566628,0.215825,-0.179164,0.999983,0.603728,0.310659,0.009625,0.999969,0.515302,...,-0.018046,0.994093,0.583166,0.914074,0.129855,0.989608,0.550327,0.910379,0.112435,0.993049
1,1.0,0.566143,0.215827,-0.166491,0.999982,0.603697,0.310681,0.032897,0.999968,0.515275,...,-0.027877,0.993640,0.583169,0.915207,0.129954,0.988759,0.549618,0.911830,0.104301,0.992580
2,1.0,0.565114,0.216485,-0.167129,0.999982,0.603651,0.311274,0.027464,0.999966,0.515112,...,-0.026802,0.993451,0.583109,0.915970,0.136486,0.988204,0.549167,0.912555,0.112636,0.992348
3,1.0,0.564665,0.218071,-0.178488,0.999981,0.603564,0.312767,0.024608,0.999965,0.514824,...,-0.027260,0.993313,0.582990,0.915961,0.139435,0.987626,0.549021,0.912643,0.116197,0.992105
4,1.0,0.564257,0.219991,-0.219889,0.999980,0.603501,0.314149,-0.001277,0.999963,0.514392,...,-0.022762,0.993191,0.582212,0.915911,0.152998,0.986772,0.548943,0.912549,0.134366,0.991728
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7580,2.0,0.512599,0.170784,-0.256588,0.999996,0.565641,0.282332,-0.039706,0.999980,0.463723,...,-0.012392,0.998173,0.611001,0.914221,0.182832,0.996932,0.415025,0.918326,0.195832,0.997662
7581,2.0,0.512604,0.170828,-0.258020,0.999996,0.565666,0.282348,-0.040138,0.999979,0.463728,...,-0.010829,0.998174,0.611088,0.914725,0.182786,0.996924,0.415027,0.918701,0.195895,0.997670
7582,2.0,0.512658,0.170774,-0.257240,0.999996,0.565671,0.282355,-0.039940,0.999979,0.463732,...,-0.010219,0.998168,0.611137,0.914791,0.182662,0.996904,0.415029,0.918997,0.195525,0.997665
7583,2.0,0.512583,0.170419,-0.255323,0.999996,0.565675,0.282372,-0.039689,0.999979,0.463743,...,-0.010150,0.998158,0.611143,0.914750,0.182363,0.996883,0.415024,0.919006,0.195029,0.997654


# 3. Lỗi gối hẹp


In [84]:
#import model để dự đoán giai đoạn
import pickle


with open("LR_Up_Down_model.pkl", "rb") as f:
    trained_model = pickle.load(f)

In [85]:
#Xử lý từng video để tính khoảng cách chân, vai
def detection_3_error(Video_folder = "Data/train/Dau_goi_hep" , Video_name = "20250228_092009000_iOS.mp4" , type = "train"):
    
    Cap = cv2.VideoCapture(f"{Video_folder}/{Video_name}")

    images = []  # Danh sách lưu hình ảnh
    max_images = 10  # Số hình ảnh tối đa hiển thị cùng lúc

    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while Cap.isOpened():
            ret, image = Cap.read()

            if not ret:
                break

            # Chuyển ảnh sang RGB
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = rescale_frame(image, percent=50)
            image.flags.writeable = True  # Make the image writable

            #trích xuất kpkp
            results = pose.process(image)

            # Kiểm tra có nhận được keypoint không
            if not results.pose_landmarks:
                continue
            
            # Chuyển lại ảnh sang BGR để hiển thị
            # image.flags.writeable = True
            # image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            #khởi tạo biến tính khoảng cách
            knee_width = feet_width = None
            
            try:


                # Tinhs toán và so sánh khoảng cách đầu gối, chân
                landmarks = results.pose_landmarks.landmark

                # Lấy trạng thái hiện tại 
                # predicted_class = trained_model.predict(X)[0]
                # prediction_probability = trained_model.predict_proba(X)[0]

                # current_stage ="middle"
                # if prediction_probability[prediction_probability.argmax()] >= 0.7: # Nếu xác suất dự đoán lớn hơn 0.5 thì mới cập nhật trạng thái
                #     if predicted_class == 0:
                #         current_stage = "down"
                #     elif current_stage == "middle" and predicted_class == 1:
                #         current_stage = "up"
                # else:
                #     current_stage = "middle"

                # print(f"Predicted class: {predicted_class}, Probability: {prediction_probability}, Current stage: {current_stage}")
            
                # Khoảng cách giữa 2 đầu gối 
                left_knee  = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
                right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]

                knee_width = calculate_distance(left_knee, right_knee)

                # Khoảng cách giữa 2 chân
                left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
                right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]

                feet_width = calculate_distance(left_ankle, right_ankle)

                # ratio = knee_width/feet_width
                # print(f"Knee width: {knee_width}, Feet width: {feet_width}, Ratio: {ratio}")
                # print("\n ------------------- \n")

                # Kiểm tra lỗi
                row = []
                # isError = False
                # if(current_stage == "down" and ratio < Min_down_ratio_knee_feet):
                #     isError = True
                # elif(current_stage == "middle" and ratio < Min_middle_ratio_knee_feet):
                #     isError = True


                # if isError:
                row.append(3)
                # else :
                #     row.append(0)
                
                # Lưu dữ liệu
                for lm in Important_kp:
                        keypoint = landmarks[mp_pose.PoseLandmark[lm].value]
                        row.extend([keypoint.x, keypoint.y, keypoint.z, keypoint.visibility])
                
                # row.extend([Video_name])
                # Kiểm tra số lượng phần tử trong row và số lượng cột trong df
                print(f"Số lượng phần tử trong row: {len(row)}")
                print(f"Số lượng cột trong df: {len(df.columns)}")

                if len(row) == len(df.columns):
                    if type == "train":
                        df.loc[len(df)] = row
                    elif type == "test":
                        df_test.loc[len(df_test)] = row
                    elif type == "validate":
                        df_va.loc[len(df_va)] = row
                else:
                    print(f"Lỗi! row có {len(row)} phần tử nhưng df có {len(df.columns)} cột.")


                # Nền
                cv2.rectangle(image, (0, 0), (500, 60), (245, 117, 16), -1)

                # Display feet distance
                cv2.putText(image, "FEET", (15, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(feet_width, 2)), (10, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                # Display knee distance
                cv2.putText(image, "KNEE", (95, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(knee_width, 2)), (90, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                # Display label
                cv2.putText(image, "LABEL", (200, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(row[0], 2)), (200, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                # Display current stage
                cv2.putText(image, "STAGE", (300, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                # cv2.putText(image, current_stage, (300, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)


            except Exception as e:
                print(f"Error: {e}")

            # Draw landmarks and connections
            # mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, mp_drawing.DrawingSpec(color=(244, 117, 66), thickness=2, circle_radius=4), mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

            images.append(image)

            # Khi đủ 10 ảnh, hiển thị tất cả cùng lúc
            # if len(images) == max_images:
            #     fig, axes = plt.subplots(5, 2, figsize=(32, 18))  # Tạo grid 2x5
            #     for i, ax in enumerate(axes.flat):
            #         ax.imshow(images[i])
            #         ax.set_aspect('auto')
            #         ax.axis("off")  # Ẩn trục tọa độ
            #     plt.show()
            #     images = []  # Reset danh sách sau khi hiển thị


        Cap.release()

detection_3_error()

In [86]:
import os

# df = pd.DataFrame(columns=header)
# df_test = pd.DataFrame(columns=header)
# df_va = pd.DataFrame(columns=header)

#Path
video_folder = "Data/Train/Dau_goi_hep"
# video_folder = "Data/train/Correct"
video_files = [f for f in os.listdir(video_folder) if f.endswith(".mp4")]

for video_file in video_files:
    detection_3_error(video_folder, video_file, "train")

video_folder = "Data/Test/Dau_goi_hep"
# video_folder = "Data/test/Correct"
video_files = [f for f in os.listdir(video_folder) if f.endswith(".mp4")]

for video_file in video_files:
    detection_3_error(video_folder, video_file, "test")

video_folder = "Data/Validation/Dau_goi_hep"
# video_folder = "Data/validate/Correct"
video_files = [f for f in os.listdir(video_folder) if f.endswith(".mp4")]

for video_file in video_files:
    detection_3_error(video_folder, video_file, "validate")

Số lượng phần tử trong row: 37
Số lượng cột trong df: 37
Số lượng phần tử trong row: 37
Số lượng cột trong df: 37
Số lượng phần tử trong row: 37
Số lượng cột trong df: 37
Số lượng phần tử trong row: 37
Số lượng cột trong df: 37
Số lượng phần tử trong row: 37
Số lượng cột trong df: 37
Số lượng phần tử trong row: 37
Số lượng cột trong df: 37
Số lượng phần tử trong row: 37
Số lượng cột trong df: 37
Số lượng phần tử trong row: 37
Số lượng cột trong df: 37
Số lượng phần tử trong row: 37
Số lượng cột trong df: 37
Số lượng phần tử trong row: 37
Số lượng cột trong df: 37
Số lượng phần tử trong row: 37
Số lượng cột trong df: 37
Số lượng phần tử trong row: 37
Số lượng cột trong df: 37
Số lượng phần tử trong row: 37
Số lượng cột trong df: 37
Số lượng phần tử trong row: 37
Số lượng cột trong df: 37
Số lượng phần tử trong row: 37
Số lượng cột trong df: 37
Số lượng phần tử trong row: 37
Số lượng cột trong df: 37
Số lượng phần tử trong row: 37
Số lượng cột trong df: 37
Số lượng phần tử trong row: 37


In [87]:
df 

,label,NOSE_x,NOSE_y,NOSE_z,NOSE_visibility,LEFT_SHOULDER_x,LEFT_SHOULDER_y,LEFT_SHOULDER_z,LEFT_SHOULDER_visibility,RIGHT_SHOULDER_x,...,RIGHT_KNEE_z,RIGHT_KNEE_visibility,LEFT_ANKLE_x,LEFT_ANKLE_y,LEFT_ANKLE_z,LEFT_ANKLE_visibility,RIGHT_ANKLE_x,RIGHT_ANKLE_y,RIGHT_ANKLE_z,RIGHT_ANKLE_visibility
0,1.0,0.566628,0.215825,-0.179164,0.999983,0.603728,0.310659,0.009625,0.999969,0.515302,...,-0.018046,0.994093,0.583166,0.914074,0.129855,0.989608,0.550327,0.910379,0.112435,0.993049
1,1.0,0.566143,0.215827,-0.166491,0.999982,0.603697,0.310681,0.032897,0.999968,0.515275,...,-0.027877,0.993640,0.583169,0.915207,0.129954,0.988759,0.549618,0.911830,0.104301,0.992580
2,1.0,0.565114,0.216485,-0.167129,0.999982,0.603651,0.311274,0.027464,0.999966,0.515112,...,-0.026802,0.993451,0.583109,0.915970,0.136486,0.988204,0.549167,0.912555,0.112636,0.992348
3,1.0,0.564665,0.218071,-0.178488,0.999981,0.603564,0.312767,0.024608,0.999965,0.514824,...,-0.027260,0.993313,0.582990,0.915961,0.139435,0.987626,0.549021,0.912643,0.116197,0.992105
4,1.0,0.564257,0.219991,-0.219889,0.999980,0.603501,0.314149,-0.001277,0.999963,0.514392,...,-0.022762,0.993191,0.582212,0.915911,0.152998,0.986772,0.548943,0.912549,0.134366,0.991728
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9691,3.0,0.411157,0.341554,-0.176379,0.999701,0.450115,0.415452,-0.041591,0.997917,0.379000,...,-0.186273,0.996643,0.457696,0.825902,0.076069,0.982477,0.398919,0.833747,-0.001992,0.993165
9692,3.0,0.411103,0.334807,-0.188470,0.999686,0.450111,0.405549,-0.043048,0.997838,0.378616,...,-0.184874,0.996673,0.457697,0.825827,0.104293,0.982412,0.398655,0.833916,0.000922,0.993331
9693,3.0,0.410970,0.329080,-0.197278,0.999674,0.450070,0.397661,-0.051085,0.997778,0.378346,...,-0.169075,0.996731,0.457683,0.824541,0.147916,0.982409,0.398547,0.834080,0.017758,0.993556
9694,3.0,0.410637,0.321747,-0.204745,0.999680,0.449970,0.390810,-0.061789,0.997819,0.377985,...,-0.116517,0.996677,0.457563,0.824258,0.138047,0.982539,0.398592,0.835221,0.056217,0.993624


# 4.Lỗi xuống quá xâu

In [88]:
#Xử lý từng video để tính khoảng cách chân, đàu gốin 
def detection_4_error(Video_folder, Video_name, type):
    
    Cap = cv2.VideoCapture(f"{Video_folder}/{Video_name}")

    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while Cap.isOpened():
            ret, image = Cap.read()

            if not ret:
                break

            # Chuyển ảnh sang RGB
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = rescale_frame(image, percent=50)
            image.flags.writeable = True  # Make the image writable

            #trích frame như ảnh đầu vào
            results = pose.process(image)

            # Kiểm tra có nhận được keypoint không
            if not results.pose_landmarks:
                continue
            
            # Chuyển lại ảnh sang BGR để hiển thị
            # image.flags.writeable = True
            # image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            #khởi tạo biến tính
            knee_angle = None

            
            try:

                # Tinhs toán và so sánh góc giữa 2 đầu gối
                landmarks = results.pose_landmarks.landmark

                # Góc đầu gối trái 
                left_knee  = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].z]
                left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].z]
                left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].z]
                left_knee_angle = calculate_angle(left_hip, left_knee, left_ankle)

                # Góc đầu gối phải
                right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].z]
                right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].z]
                right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].z]

                right_knee_angle = calculate_angle(right_hip, right_knee, right_ankle)

                #Láy trung bình cộng 2 góc
                knee_angle = (left_knee_angle + right_knee_angle) / 2

                row = []
                if(knee_angle < Min_knee_angle):
                    row.append(4)
                # else:
                #     row.append(0)
                
                # Lưu dữ liệu
                for lm in Important_kp:
                        keypoint = landmarks[mp_pose.PoseLandmark[lm].value]
                        row.extend([keypoint.x, keypoint.y, keypoint.z, keypoint.visibility])
                
                # row.extend([Video_name])
                
                if len(row) == len(df.columns):
                    if type == "train":
                        df.loc[len(df)] = row
                    elif type == "test":
                        df_test.loc[len(df_test)] = row
                    elif type == "validate":
                        df_va.loc[len(df_va)] = row
                else:
                    print(f"Lỗi! row có {len(row)} phần tử nhưng df có {len(df.columns)} cột.")

                # Nền
                cv2.rectangle(image, (0, 0), (500, 60), (245, 117, 16), -1)

                #Display label
                cv2.putText(image, "LABEL", (10, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(row[0], 2)), (10, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                # Display knee angle 
                cv2.putText(image, "knee_angle", (150, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(knee_angle, 2)), (150, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

            except Exception as e:
                print(f"Error: {e}")

            # Draw landmarks and connections
            # mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, mp_drawing.DrawingSpec(color=(244, 117, 66), thickness=2, circle_radius=4), mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

            # if(knee_angle < 55):
            # plt.imshow(image)
            # plt.axis("off")  # Ẩn trục tọa độ
            # plt.show()

        Cap.release()

# Đường dẫn thư mục chứa video
# video_folder = "Data/Train/Correct"
# video_files = "20250228_150821.mp4"

# video_folder = "Data/Train/Gap_lung"
# video_files = "20250228_165326.mp4"

# video_folder = "Data/Train/Xuong_qua_xau"
# video_files = "20250228_163416.mp4"

# process_frame(video_folder, video_files)



In [89]:
import os

# df = pd.DataFrame(columns=header)
# df_test = pd.DataFrame(columns=header)
# df_va = pd.DataFrame(columns=header)

#Path
video_folder = "Data/Train/Xuong_qua_sau"
# video_folder = "Data/train/Correct"
video_files = [f for f in os.listdir(video_folder) if f.endswith(".mp4")]

for video_file in video_files:
    detection_4_error(video_folder, video_file, "train")

video_folder = "Data/Test/Xuong_qua_sau"
# video_folder = "Data/test/Correct"
video_files = [f for f in os.listdir(video_folder) if f.endswith(".mp4")]

for video_file in video_files:
    detection_4_error(video_folder, video_file, "test")

video_folder = "Data/Validation/Xuong_qua_sau"
# video_folder = "Data/validate/Correct"
video_files = [f for f in os.listdir(video_folder) if f.endswith(".mp4")]

for video_file in video_files:
    detection_4_error(video_folder, video_file, "validate")

Lỗi! row có 36 phần tử nhưng df có 37 cột.
Lỗi! row có 36 phần tử nhưng df có 37 cột.
Lỗi! row có 36 phần tử nhưng df có 37 cột.
Lỗi! row có 36 phần tử nhưng df có 37 cột.
Lỗi! row có 36 phần tử nhưng df có 37 cột.
Lỗi! row có 36 phần tử nhưng df có 37 cột.
Lỗi! row có 36 phần tử nhưng df có 37 cột.
Lỗi! row có 36 phần tử nhưng df có 37 cột.
Lỗi! row có 36 phần tử nhưng df có 37 cột.
Lỗi! row có 36 phần tử nhưng df có 37 cột.
Lỗi! row có 36 phần tử nhưng df có 37 cột.
Lỗi! row có 36 phần tử nhưng df có 37 cột.
Lỗi! row có 36 phần tử nhưng df có 37 cột.
Lỗi! row có 36 phần tử nhưng df có 37 cột.
Lỗi! row có 36 phần tử nhưng df có 37 cột.
Lỗi! row có 36 phần tử nhưng df có 37 cột.
Lỗi! row có 36 phần tử nhưng df có 37 cột.
Lỗi! row có 36 phần tử nhưng df có 37 cột.
Lỗi! row có 36 phần tử nhưng df có 37 cột.
Lỗi! row có 36 phần tử nhưng df có 37 cột.
Lỗi! row có 36 phần tử nhưng df có 37 cột.
Lỗi! row có 36 phần tử nhưng df có 37 cột.
Lỗi! row có 36 phần tử nhưng df có 37 cột.
Lỗi! row có

In [90]:
df

,label,NOSE_x,NOSE_y,NOSE_z,NOSE_visibility,LEFT_SHOULDER_x,LEFT_SHOULDER_y,LEFT_SHOULDER_z,LEFT_SHOULDER_visibility,RIGHT_SHOULDER_x,...,RIGHT_KNEE_z,RIGHT_KNEE_visibility,LEFT_ANKLE_x,LEFT_ANKLE_y,LEFT_ANKLE_z,LEFT_ANKLE_visibility,RIGHT_ANKLE_x,RIGHT_ANKLE_y,RIGHT_ANKLE_z,RIGHT_ANKLE_visibility
0,1.0,0.566628,0.215825,-0.179164,0.999983,0.603728,0.310659,0.009625,0.999969,0.515302,...,-0.018046,0.994093,0.583166,0.914074,0.129855,0.989608,0.550327,0.910379,0.112435,0.993049
1,1.0,0.566143,0.215827,-0.166491,0.999982,0.603697,0.310681,0.032897,0.999968,0.515275,...,-0.027877,0.993640,0.583169,0.915207,0.129954,0.988759,0.549618,0.911830,0.104301,0.992580
2,1.0,0.565114,0.216485,-0.167129,0.999982,0.603651,0.311274,0.027464,0.999966,0.515112,...,-0.026802,0.993451,0.583109,0.915970,0.136486,0.988204,0.549167,0.912555,0.112636,0.992348
3,1.0,0.564665,0.218071,-0.178488,0.999981,0.603564,0.312767,0.024608,0.999965,0.514824,...,-0.027260,0.993313,0.582990,0.915961,0.139435,0.987626,0.549021,0.912643,0.116197,0.992105
4,1.0,0.564257,0.219991,-0.219889,0.999980,0.603501,0.314149,-0.001277,0.999963,0.514392,...,-0.022762,0.993191,0.582212,0.915911,0.152998,0.986772,0.548943,0.912549,0.134366,0.991728
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11189,4.0,0.520547,0.490565,-0.332984,0.999982,0.559115,0.561726,-0.163108,0.999954,0.461736,...,-0.267443,0.987082,0.541947,0.873421,-0.057337,0.958818,0.463015,0.874559,-0.058330,0.985433
11190,4.0,0.520319,0.479626,-0.340029,0.999982,0.559071,0.553804,-0.168140,0.999955,0.461576,...,-0.270265,0.987740,0.542006,0.873346,-0.051844,0.960264,0.462561,0.874561,-0.055994,0.986080
11191,4.0,0.520311,0.470814,-0.344187,0.999983,0.559057,0.546626,-0.171157,0.999955,0.461426,...,-0.271075,0.988352,0.542171,0.873407,-0.048063,0.961480,0.462074,0.874411,-0.053012,0.986711
11192,4.0,0.520036,0.463128,-0.369012,0.999984,0.559025,0.538718,-0.186496,0.999955,0.461174,...,-0.289293,0.988929,0.542320,0.873399,-0.044097,0.962953,0.461968,0.874109,-0.045356,0.987130


# 5. Lỗi gập lưng


In [91]:
def detection_5_error(Video_folder, Video_name, type):
    
    Cap = cv2.VideoCapture(f"{Video_folder}/{Video_name}")

    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while Cap.isOpened():
            ret, image = Cap.read()

            if not ret:
                break

            # Chuyển ảnh sang RGB
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = rescale_frame(image, percent=50)
            image.flags.writeable = True  # Make the image writable

            #trích frame như ảnh đầu vào
            results = pose.process(image)

            # Kiểm tra có nhận được keypoint không
            if not results.pose_landmarks:
                continue
            
            # Chuyển lại ảnh sang BGR để hiển thị
            # image.flags.writeable = True
            # image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            #khởi tạo biến tính
            knee_angle = None
            hip_angle = None

            
            try:

                # Tinhs toán và so sánh góc giữa 2 đầu gối
                landmarks = results.pose_landmarks.landmark

                # Góc đầu gối trái 
                left_knee  = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].z]
                left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].z]
                left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].z]
                left_knee_angle = calculate_angle(left_hip, left_knee, left_ankle)

                # Góc đầu gối phải
                right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].z]
                right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].z]
                right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].z]

                right_knee_angle = calculate_angle(right_hip, right_knee, right_ankle)

                #Láy trung bình cộng 2 góc
                knee_angle = (left_knee_angle + right_knee_angle) / 2

                #Góc hông phải
                right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].z]
                right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].z]
                right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].z]

                right_hip_angle = calculate_angle(right_shoulder, right_hip, right_knee)

                #Góc hông trái
                left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].z]
                left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].z]
                left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].z]

                left_hip_angle = calculate_angle(left_shoulder, left_hip, left_knee)

                #Lấy trung bình cộng 2 góc
                hip_angle = (left_hip_angle + right_hip_angle) / 2

                ratio = knee_angle/hip_angle

                #Kiểm tra lỗi
                row = []
                if(ratio > Max_ratio_knee_hip):
                    row.append(5)
                # else:
                #     row.append(0)

                # Lưu dữ liệu
                for lm in Important_kp:
                        keypoint = landmarks[mp_pose.PoseLandmark[lm].value]
                        row.extend([keypoint.x, keypoint.y, keypoint.z, keypoint.visibility])
                
                # row.extend([Video_name])
                
                if len(row) == len(df.columns):
                    if type == "train":
                        df.loc[len(df)] = row
                    elif type == "test":
                        df_test.loc[len(df_test)] = row
                    elif type == "validate":
                        df_va.loc[len(df_va)] = row
                else:
                    print(f"Lỗi! row có {len(row)} phần tử nhưng df có {len(df.columns)} cột.")

                # Nền
                cv2.rectangle(image, (0, 0), (500, 60), (245, 117, 16), -1)

                # Display hip angle
                cv2.putText(image, "hip_angle", (15, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(hip_angle, 2)), (10, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                # Display knee angle 
                cv2.putText(image, "knee_ankle", (150, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(knee_angle, 2)), (150, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

            except Exception as e:
                print(f"Error: {e}")

            # Draw landmarks and connections
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, mp_drawing.DrawingSpec(color=(244, 117, 66), thickness=2, circle_radius=4), mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

            # Hiển thị ảnh
            # if(hip_angle/shoulder_hip_ankle_angle <0.52):
            #     plt.imshow(image)
            #     plt.axis("off")  # Ẩn trục tọa độ
            #     plt.show()

            # plt.imshow(image)
            # plt.axis("off")  # Ẩn trục tọa độ
            # plt.show()

        Cap.release()

In [92]:
import os

# df = pd.DataFrame(columns=header)
# df_test = pd.DataFrame(columns=header)
# df_va = pd.DataFrame(columns=header)

#Path
# video_folder = "Data/train/Xuong_qua_xau"
video_folder = "Data/Train/Gap_lung"
# video_folder = "Data/train/Correct"
video_files = [f for f in os.listdir(video_folder) if f.endswith(".mp4")]

for video_file in video_files:
    detection_5_error(video_folder, video_file, "train")

video_folder = "Data/Test/Gap_lung"
# video_folder = "Data/test/Correct"
video_files = [f for f in os.listdir(video_folder) if f.endswith(".mp4")]

for video_file in video_files:
    detection_5_error(video_folder, video_file, "test")

video_folder = "Data/Validation/Gap_lung"
# video_folder = "Data/validate/Correct"
video_files = [f for f in os.listdir(video_folder) if f.endswith(".mp4")]

for video_file in video_files:
    detection_5_error(video_folder, video_file, "validate")

Lỗi! row có 36 phần tử nhưng df có 37 cột.
Lỗi! row có 36 phần tử nhưng df có 37 cột.
Lỗi! row có 36 phần tử nhưng df có 37 cột.
Lỗi! row có 36 phần tử nhưng df có 37 cột.
Lỗi! row có 36 phần tử nhưng df có 37 cột.
Lỗi! row có 36 phần tử nhưng df có 37 cột.
Lỗi! row có 36 phần tử nhưng df có 37 cột.
Lỗi! row có 36 phần tử nhưng df có 37 cột.
Lỗi! row có 36 phần tử nhưng df có 37 cột.
Lỗi! row có 36 phần tử nhưng df có 37 cột.
Lỗi! row có 36 phần tử nhưng df có 37 cột.
Lỗi! row có 36 phần tử nhưng df có 37 cột.
Lỗi! row có 36 phần tử nhưng df có 37 cột.
Lỗi! row có 36 phần tử nhưng df có 37 cột.
Lỗi! row có 36 phần tử nhưng df có 37 cột.
Lỗi! row có 36 phần tử nhưng df có 37 cột.
Lỗi! row có 36 phần tử nhưng df có 37 cột.
Lỗi! row có 36 phần tử nhưng df có 37 cột.
Lỗi! row có 36 phần tử nhưng df có 37 cột.
Lỗi! row có 36 phần tử nhưng df có 37 cột.
Lỗi! row có 36 phần tử nhưng df có 37 cột.
Lỗi! row có 36 phần tử nhưng df có 37 cột.
Lỗi! row có 36 phần tử nhưng df có 37 cột.
Lỗi! row có

In [93]:
df

,label,NOSE_x,NOSE_y,NOSE_z,NOSE_visibility,LEFT_SHOULDER_x,LEFT_SHOULDER_y,LEFT_SHOULDER_z,LEFT_SHOULDER_visibility,RIGHT_SHOULDER_x,...,RIGHT_KNEE_z,RIGHT_KNEE_visibility,LEFT_ANKLE_x,LEFT_ANKLE_y,LEFT_ANKLE_z,LEFT_ANKLE_visibility,RIGHT_ANKLE_x,RIGHT_ANKLE_y,RIGHT_ANKLE_z,RIGHT_ANKLE_visibility
0,1.0,0.566628,0.215825,-0.179164,0.999983,0.603728,0.310659,0.009625,0.999969,0.515302,...,-0.018046,0.994093,0.583166,0.914074,0.129855,0.989608,0.550327,0.910379,0.112435,0.993049
1,1.0,0.566143,0.215827,-0.166491,0.999982,0.603697,0.310681,0.032897,0.999968,0.515275,...,-0.027877,0.993640,0.583169,0.915207,0.129954,0.988759,0.549618,0.911830,0.104301,0.992580
2,1.0,0.565114,0.216485,-0.167129,0.999982,0.603651,0.311274,0.027464,0.999966,0.515112,...,-0.026802,0.993451,0.583109,0.915970,0.136486,0.988204,0.549167,0.912555,0.112636,0.992348
3,1.0,0.564665,0.218071,-0.178488,0.999981,0.603564,0.312767,0.024608,0.999965,0.514824,...,-0.027260,0.993313,0.582990,0.915961,0.139435,0.987626,0.549021,0.912643,0.116197,0.992105
4,1.0,0.564257,0.219991,-0.219889,0.999980,0.603501,0.314149,-0.001277,0.999963,0.514392,...,-0.022762,0.993191,0.582212,0.915911,0.152998,0.986772,0.548943,0.912549,0.134366,0.991728
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12584,5.0,0.502140,0.413816,-0.473057,0.999984,0.546305,0.430141,-0.230609,0.999992,0.452828,...,-0.158037,0.991018,0.543855,0.900754,-0.048971,0.973336,0.457092,0.896660,-0.022327,0.990995
12585,5.0,0.502404,0.397786,-0.456841,0.999981,0.546309,0.419557,-0.214265,0.999989,0.453119,...,-0.166889,0.991491,0.543796,0.900634,-0.033763,0.974921,0.457090,0.896852,-0.017943,0.991395
12586,5.0,0.503870,0.380777,-0.428196,0.999976,0.546271,0.409370,-0.190015,0.999986,0.453118,...,-0.151754,0.991962,0.543606,0.900461,-0.033545,0.976584,0.457072,0.896918,-0.013037,0.991832
12587,5.0,0.503971,0.366431,-0.401071,0.999970,0.546266,0.398942,-0.168638,0.999982,0.453083,...,-0.130439,0.992415,0.543525,0.900597,-0.027711,0.978049,0.457071,0.897068,-0.004526,0.992231


# 6. Thêm data đúng

In [94]:
def detection_correct(Video_folder, Video_name, type):
    
    Cap = cv2.VideoCapture(f"{Video_folder}/{Video_name}")

    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while Cap.isOpened():
            ret, image = Cap.read()

            if not ret:
                break

            # Chuyển ảnh sang RGB
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = rescale_frame(image, percent=50)
            image.flags.writeable = True  # Make the image writable

            #trích frame như ảnh đầu vào
            results = pose.process(image)

            # Kiểm tra có nhận được keypoint không
            if not results.pose_landmarks:
                continue
            
            # Chuyển lại ảnh sang BGR để hiển thị
            # image.flags.writeable = True
            # image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            #khởi tạo biến tính
            knee_angle = None
            hip_angle = None

            
            try:
                # Lấy kp từ frame
                landmarks = results.pose_landmarks.landmark

                #Kiểm tra lỗi
                row = []

                row.append(0)
                for lm in Important_kp:
                        keypoint = landmarks[mp_pose.PoseLandmark[lm].value]
                        row.extend([keypoint.x, keypoint.y, keypoint.z, keypoint.visibility])
                
                # row.extend([Video_name])
                
                if len(row) == len(df.columns):
                    if type == "train":
                        df.loc[len(df)] = row
                    elif type == "test":
                        df_test.loc[len(df_test)] = row
                    elif type == "validate":
                        df_va.loc[len(df_va)] = row
                else:
                    print(f"Lỗi! row có {len(row)} phần tử nhưng df có {len(df.columns)} cột.")


            except Exception as e:
                print(f"Error: {e}")

            # Draw landmarks and connections
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, mp_drawing.DrawingSpec(color=(244, 117, 66), thickness=2, circle_radius=4), mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

            # Hiển thị ảnh
            # if(hip_angle/shoulder_hip_ankle_angle <0.52):
            #     plt.imshow(image)
            #     plt.axis("off")  # Ẩn trục tọa độ
            #     plt.show()

            # plt.imshow(image)
            # plt.axis("off")  # Ẩn trục tọa độ
            # plt.show()

        Cap.release()


In [95]:
import os

#Path
video_folder = "Data/Train/Correct"
video_files = [f for f in os.listdir(video_folder) if f.endswith(".mp4")]

for video_file in video_files:
    detection_correct(video_folder, video_file, "train")

video_folder = "Data/Test/Correct"
video_files = [f for f in os.listdir(video_folder) if f.endswith(".mp4")]

for video_file in video_files:
    detection_correct(video_folder, video_file, "test")

video_folder = "Data/Validation/Correct"
video_files = [f for f in os.listdir(video_folder) if f.endswith(".mp4")]

for video_file in video_files:
    detection_correct(video_folder, video_file, "validate")


In [96]:
df

,label,NOSE_x,NOSE_y,NOSE_z,NOSE_visibility,LEFT_SHOULDER_x,LEFT_SHOULDER_y,LEFT_SHOULDER_z,LEFT_SHOULDER_visibility,RIGHT_SHOULDER_x,...,RIGHT_KNEE_z,RIGHT_KNEE_visibility,LEFT_ANKLE_x,LEFT_ANKLE_y,LEFT_ANKLE_z,LEFT_ANKLE_visibility,RIGHT_ANKLE_x,RIGHT_ANKLE_y,RIGHT_ANKLE_z,RIGHT_ANKLE_visibility
0,1.0,0.566628,0.215825,-0.179164,0.999983,0.603728,0.310659,0.009625,0.999969,0.515302,...,-0.018046,0.994093,0.583166,0.914074,0.129855,0.989608,0.550327,0.910379,0.112435,0.993049
1,1.0,0.566143,0.215827,-0.166491,0.999982,0.603697,0.310681,0.032897,0.999968,0.515275,...,-0.027877,0.993640,0.583169,0.915207,0.129954,0.988759,0.549618,0.911830,0.104301,0.992580
2,1.0,0.565114,0.216485,-0.167129,0.999982,0.603651,0.311274,0.027464,0.999966,0.515112,...,-0.026802,0.993451,0.583109,0.915970,0.136486,0.988204,0.549167,0.912555,0.112636,0.992348
3,1.0,0.564665,0.218071,-0.178488,0.999981,0.603564,0.312767,0.024608,0.999965,0.514824,...,-0.027260,0.993313,0.582990,0.915961,0.139435,0.987626,0.549021,0.912643,0.116197,0.992105
4,1.0,0.564257,0.219991,-0.219889,0.999980,0.603501,0.314149,-0.001277,0.999963,0.514392,...,-0.022762,0.993191,0.582212,0.915911,0.152998,0.986772,0.548943,0.912549,0.134366,0.991728
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17407,0.0,0.506316,0.130152,-0.281515,0.999983,0.552153,0.248577,-0.063084,0.999896,0.455341,...,0.005616,0.996638,0.548200,0.868649,0.186076,0.994231,0.458890,0.873478,0.156291,0.996436
17408,0.0,0.506389,0.125906,-0.276450,0.999983,0.551215,0.245856,-0.071266,0.999893,0.454812,...,0.005915,0.996662,0.547458,0.866486,0.169546,0.994282,0.458107,0.868949,0.153682,0.996503
17409,0.0,0.506100,0.119856,-0.289413,0.999982,0.550659,0.239843,-0.089731,0.999887,0.453896,...,0.009283,0.996628,0.546583,0.858204,0.180645,0.994244,0.457531,0.861787,0.154664,0.996517
17410,0.0,0.504839,0.114473,-0.284251,0.999980,0.549181,0.232904,-0.085296,0.999883,0.452999,...,0.013782,0.996513,0.544935,0.850807,0.185347,0.993994,0.455116,0.853462,0.169072,0.996392


# 7. Đưa dataframe ra csv

In [97]:
def save_splits_to_csv(train_df, test_df, val_df, output_dir="dataset"):
    """
    Lưu các DataFrame train, test, validation ra file CSV

    Args:
        train_df (pd.DataFrame): Dữ liệu train
        test_df (pd.DataFrame): Dữ liệu test
        val_df (pd.DataFrame): Dữ liệu validation
        output_dir (str): Thư mục lưu file CSV (mặc định là 'output')
    """
    import os
    os.makedirs(output_dir, exist_ok=True)  # Tạo thư mục nếu chưa có
    
    train_df.to_csv(f"{output_dir}/train.csv", index=False)
    test_df.to_csv(f"{output_dir}/test.csv", index=False)
    val_df.to_csv(f"{output_dir}/validation.csv", index=False)
    
    print(f"Dữ liệu đã được lưu vào thư mục '{output_dir}'.")

In [98]:
save_splits_to_csv(df, df_test, df_va, "dataset")

Dữ liệu đã được lưu vào thư mục 'dataset'.
